In [1]:
import pandas as pd

C:\Users\Lucie GALLAND\AppData\Local\Temp\ipykernel_17628\2035458946.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
input_path = '...'
name_data = 'hope_test.jsonl'
df = pd.read_csv(input_path)

In [5]:
df['last_utterance'] = df['Utterance'].shift(1)
df['last_id'] = df['ID'].shift(1)
df['two_utterance_from_same_conv'] = df.apply(lambda w :   w['ID'].split('_')[0] == w['last_id'].split('_')[0] if w['last_id'] is not None else False ,axis=1)
df['last_utterance'] = df.apply(lambda x: x['last_utterance'] if x['two_utterance_from_same_conv'] else "", axis=1)
df['interlocutor'] = df['Type'].apply(lambda x: "Therapist: " if x == "T" else "Patient: ")
df['last_interlocutor'] = df['interlocutor'].shift(1)
df['last_interlocutor'] = df.apply(lambda x: x['last_interlocutor'] if x['two_utterance_from_same_conv'] else "", axis=1)
df['utterance_and_context'] = df['last_interlocutor'] + df['last_utterance']+" " + df['interlocutor'] + df['Utterance']

In [6]:
df['two_last_utterance'] = df['utterance_and_context'].shift(2)
df['two_last_id'] = df['ID'].shift(2)
df['two_utterance_from_same_conv'] = df.apply(lambda w :   w['ID'].split('_')[0] == w['two_last_id'].split('_')[0] if w['two_last_id'] is not None else False ,axis=1)
df['two_last_utterance'] = df.apply(lambda x: x['two_last_utterance'] if x['two_utterance_from_same_conv'] else "", axis=1)
df['interlocutor'] = df['Type'].apply(lambda x: "Therapist: " if x == "T" else "Patient: ")
df['two_last_interlocutor'] = df['interlocutor'].shift(2)
df['two_last_interlocutor'] = df.apply(lambda x: x['two_last_interlocutor'] if x['two_utterance_from_same_conv'] else "", axis=1)
df['context'] =  df['two_last_utterance']+" "  + df['last_interlocutor'] + df['last_utterance']

In [7]:
df[['ID', 'Utterance','context','Type','Dialog_Act']].to_json('dataset/'+name_data)